In [82]:
import datetime
import pandas as pd

class Record(pd.DataFrame):
    
    def __init__(self, nutrition):
        self.__history = nutrition
        
    def add(self, date, protein, carbon, fat):
        calorie = protein/4 + carbon/4 + fat/9
        new_record = pd.Series({"Date": pd.to_datetime(date), "Calorie": calorie, "Protein": protein, "Carbon": carbon, "Fat": fat})
        self.__history.append(new_record)
        
    def remove(self, date):
        print("Are you sure you want to delete the following record?\n",self.__history[self.__history["Date"]==pd.to_datetime(datetime)])
        while True:
            check = input("""Enter [1] for "YES"/ [2] for "NO" ("q" to quit):""")
            if check == "1":
                break
            elif check == "2" or check == "q":
                return None
            else:
                print("Invalid input, please re-enter")
                continue
                
        self.__history = self.__history[~(self.__history["Date"] == pd.to_datetime(date))]
    